# Implementação do Dilithium

## Parameters

In [1]:
n = 256
q = 8380417
d = 14
peso = 60
y1 = (q-1)/16
y2 = y1/2
k = 3
l = 2
eta = 7
beta = 375
omega = 64

Zx.<x>  = ZZ[]
Gq.<z>  = PolynomialRing(GF(q))

R.<x>  = Zx.quotient(x^n+1)
Rq.<z> = Gq.quotient(z^n+1)

## Funções auxiliares

In [2]:
def sam (limit,size):
    lista = []
    for i in range(size):
        poly = []
        for j in range(n):
            poly.append(randint(1,limit))
        lista.append(Rq(poly))
    
    res = matrix(Rq,size,1,lista)
    return res

def HighBits(poly):
    lista = poly.list()
    for i in range(len(lista)):
        f = lista[i]
        F = f.list()
        for j in range(len(F)):
            F[j] = HBAux(int(F[j]))
        
        lista[i] = F

    return lista

def LowBits(poly):
    lista = poly.list()
    for i in range(len(lista)):
        f = lista[i]
        F = f.list()
        for j in range(len(F)):
            F[j] = LBAux(int(F[j]))
        
        lista[i] = F

    return lista

def HBAux(C):
    res = Decompose(C,2*y2)
    return res[0]

def LBAux(C):
    res = Decompose(C,2*y2)
    return res[1]

def Decompose(C,alfa):
    r = mod(C,int(q))
    r0 = int(mod(r, int(alfa)))

    if (r-r0) == (q-1):
        r1 = 0
        r0 = r0 - 1
    else:
        r1 = (r-r0)/(int(alfa))

    return (r1,r0)

def normaI(v):
    for i in range(v.nrows()):
        norma = normaIAux(v[i],q)
        v[i] = norma
    return max(v)

def normaIAux(poly,number):
    lista = poly.list()
    for i in range(len(lista)):
        f = lista[i]
        F = f.list()
        for j in range(len(F)):
            F[j] = abs(int(F[j]))
        lista[i]=F

    List = []
    for i in range(len(lista)):
        List.append(max(lista[i]))
    
    return max(List)

def Hash(value):
    H = []
    contador = 0
    contador_ = 0
    for i in range(0,n,2):
        u=value[i]+value[i+1]
        contador = contador + 1
        if u == '11':
            H.append(0)
        if u == '01':
            H.append(1)
            contador_ = contador_ + 1
        if u == '00':
            pass
        if u == '10':
            H.append(-1)
            contador_ = contador_ + 1
        if contador_ >= peso:
            break
    
    for i in range(n-contador):
        H.append(0)
    
    return H

## Key generation, Sign and Verify

In [3]:
import hashlib

def KeyGen():
    fillA = []
    for i in range(k*l):
        fillA.append(Rq.random_element())
    
    A = matrix(Rq,k,l,fillA)
    
    s1 = sam(eta,l)
    s2 = sam(eta,k)
    
    t = A*s1 + s2
    
    pk = (A,t)
    sk = (s1,s2)
    return {'pk': pk,'sk': sk }

def Sign(text,keys):
    A = keys['pk'][0]
    s1 = keys['sk'][0]
    s2 = keys['sk'][1]
    
    y = sam(y1-1, l)
    Ay = A*y
    
    w = HighBits(Ay)
    
    string = ''
    string = string + text[2:]
    
    for i in range(len(w)):
        for j in range(len(w[i])):
            k = bin(w[i][j])
            if w[i][j] >= 0:
                string = string + k[2:]
            if w[i][j] < 0:
                string = string + k[3:]
    
    c = Hash(string)
    cQ = Rq(c)
    
    z = y + cQ*s1
 
    while (int(normaI(z)[0])) >= (y1-beta) and (normaI(LowBits(Ay-cQ*s2))) >= (y2-beta):
        y = sam(y1-1, l)
        Ay = A*y

        w = HighBits(Ay)

        string = ''
        string = string + text[2:]

        for i in range(len(w)):
            for j in range(len(w[i])):
                k = bin(w[i][j])
                if w[i][j] >= 0:
                    string = string + k[2:]
                if w[i][j] < 0:
                    string = string + k[3:]

        c = Hash(string)
        cQ = Rq(c)

        z = y + cQ*s1

    return {'z': z, 'c': cQ}

def Verify(text,cripto,keys):
    A = keys['pk'][0]
    t = keys['pk'][1]

    z = cripto['z']
    c = cripto['c']
    
    cQ = Rq(c)
    
    w = HighBits(A*z-cQ*t)
    
    string = ''
    string = string + text[2:]

    for i in range(len(w)):
        for j in range(len(w[i])):
            k = bin(w[i][j])
            if w[i][j] >= 0:
                string = string + k[2:]
            if w[i][j] < 0:
                string = string + k[3:]
    
    hashC = Hash(string) 
    
    print hashC==c
    print (int(normaI(z)[0])) < (y1-beta)

    if (int(normaI(z)[0])) < (y1-beta) or hashC != c:
        print 'Denied'
        return -1
    else:
        print 'All good'
        return 1

## Test

In [4]:
import base64

def run():
    keys = KeyGen()
    text = bin(1024)
    cripto = Sign(text,keys)
    res = Verify(text,cripto,keys)
    return res

In [5]:
run()

False
True
Denied


-1